Libraries

In [71]:
from bs4 import BeautifulSoup
import os
import json
import csv
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [35]:
import requests

url ='https://unsplash.com/'
Headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5'
}

# Making a GET request
response= requests.get(url, headers=Headers)
print(response.status_code)


200


In [36]:
soup = BeautifulSoup(response.text , 'html.parser')
print(soup.title)

<title data-rh="true">Beautiful Free Images &amp; Pictures | Unsplash</title>


In [37]:
print(soup.prettify()[:500])

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <title data-rh="true">
   Beautiful Free Images &amp; Pictures | Unsplash
  </title>
  <link data-rh="true" href="https://unsplash.com/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link data-rh="true" href="https://unsplash.com/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link data-rh="true" href="https://unsplash.com/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link color="#000000" data-r


In [72]:
# Find all img tags
img_tags = soup.find_all('img')

# Extract the image links (considering srcset as well)
image_links = []
for img in img_tags:
    if 'srcset' in img.attrs:
        # srcset can have multiple image URLs, usually we take the first one
        srcset = img['srcset'].split(',')[0]
        image_links.append(srcset.split(' ')[0])
    elif 'src' in img.attrs:
        image_links.append(img['src'])

# Print the number of image links found and the first few links for verification
print(f'Found {len(image_links)} image links.')
print(image_links[:5])  # Print the first 5 image links for verification


Found 195 image links.
['https://unsplash-assets.imgix.net/unsplashplus/asset-plus-017.jpg?dpr=1&h=280&w=280&crop=focalpoint&fp-y=0.4&auto=format&fit=crop&q=60', 'https://unsplash-assets.imgix.net/unsplashplus/asset-plus-018.jpg?dpr=1&h=280&w=280&crop=focalpoint&fp-y=0.51&auto=format&fit=crop&q=60', 'https://unsplash-assets.imgix.net/unsplashplus/asset-plus-004.jpg?dpr=1&h=280&w=280&crop=faces&auto=format&fit=crop&q=60', 'https://unsplash-assets.imgix.net/unsplashplus/asset-plus-005.jpg?dpr=1&h=280&w=280&crop=faces&auto=format&fit=crop&q=60', 'https://unsplash-assets.imgix.net/unsplashplus/asset-plus-017.jpg?dpr=1&h=280&w=280&auto=format&fit=crop&q=60']


In [73]:
# Function to extract image URLs from the webpage
def extract_image_urls(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        img_tags = soup.find_all('img')
        image_urls = []
        for img in img_tags:
            if 'srcset' in img.attrs:
                srcset = img['srcset'].split(',')[0]
                image_urls.append(srcset.split(' ')[0])
            elif 'src' in img.attrs:
                image_urls.append(img['src'])
        return image_urls
    else:
        print(f"Failed to fetch page: {url}")
        return []

In [74]:
# Function to download images without metadata
def download_images(image_urls, download_folder):
    os.makedirs(download_folder, exist_ok=True)
    for num, url in enumerate(image_urls):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                img_name = f'image_{num}.jpg'
                img_path = os.path.join(download_folder, img_name)
                with open(img_path, 'wb') as f:
                    f.write(response.content)
                print(f'Downloaded {img_name}')
            else:
                print(f"Failed to download {url}: Status code {response.status_code}")
        except Exception as e:
            print(f"Error occurred while downloading {url}: {e}")


In [75]:
# Function to save image URLs to a CSV file
def save_urls_to_csv(image_urls, csv_file):
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Image URL'])
        for url in image_urls:
            writer.writerow([url])
    print(f"Image URLs saved to {csv_file}")

In [77]:
# Extract image URLs from the webpage
image_urls = extract_image_urls(url)

# Download images without metadata
download_folder = 'unsplash_images'
download_images(image_urls, download_folder)



Downloaded image_0.jpg
Downloaded image_1.jpg
Downloaded image_2.jpg
Downloaded image_3.jpg
Downloaded image_4.jpg
Downloaded image_5.jpg
Downloaded image_6.jpg
Downloaded image_7.jpg
Downloaded image_8.jpg
Downloaded image_9.jpg
Downloaded image_10.jpg
Downloaded image_11.jpg
Downloaded image_12.jpg
Downloaded image_13.jpg
Downloaded image_14.jpg
Downloaded image_15.jpg
Downloaded image_16.jpg
Downloaded image_17.jpg
Downloaded image_18.jpg
Downloaded image_19.jpg
Downloaded image_20.jpg
Downloaded image_21.jpg
Downloaded image_22.jpg
Downloaded image_23.jpg
Downloaded image_24.jpg
Error occurred while downloading : No connection adapters were found for ''
Downloaded image_26.jpg
Downloaded image_27.jpg
Error occurred while downloading : No connection adapters were found for ''
Downloaded image_29.jpg
Downloaded image_30.jpg
Error occurred while downloading : No connection adapters were found for ''
Downloaded image_32.jpg
Downloaded image_33.jpg
Error occurred while downloading : No 

In [78]:
# Save image URLs to a CSV file
csv_file = 'image_urls.csv'
save_urls_to_csv(image_urls, csv_file)

Image URLs saved to image_urls.csv


In [39]:
# Create directory for images
os.makedirs('unsplash_images', exist_ok=True)

In [63]:
# Replace with your actual PhotoTag.ai API key
PHOTO_TAG_API_KEY = 'WMVK-z0DR-HabZ-OSjuP'
PHOTO_TAG_API_URL = 'https://server.phototag.ai/api/keywords'


def get_tags_from_phototag(image_path):
    try:
        with open(image_path, 'rb') as img_file:
            payload = {
                "language": "en",
                "maxKeywords": 5,
                "requiredKeywords": "beach,sky",  # Adjust as needed
                "customContext": "vacation photo"  # Adjust as needed
            }
            files = {'file': ('image.jpg', img_file)}
            headers = {
                "Authorization": f'Bearer {"WMVK-z0DR-HabZ-OSjuP"}'
            }
            response = requests.post(
                "https://server.phototag.ai/api/keywords",
                headers=headers,
                data=payload,
                files=files
            )
            if response.status_code == 200:
                metadata = response.json()
                return metadata
            else:
                print(f"Failed to get tags for {image_path}: Status code {response.status_code}")
                return {'tags': []}
    except Exception as e:
        print(f"Error occurred while getting tags for {image_path}: {e}")
        return {'tags': []}

# Download images and get metadata from PhotoTag.ai
for num, link in enumerate(image_links):
    try:
        response = requests.get(link)
        if response.status_code == 200:
            img_name = f'image_{num}.jpg'
            img_path = os.path.join('unsplash_images', img_name)
            with open(img_path, 'wb') as f:
                f.write(response.content)
            print(f'Downloaded {img_name}')

            # Get metadata from PhotoTag.ai
            metadata = get_tags_from_phototag(img_path)
            metadata.update({
                'image_name': img_name,
                'image_path': img_path,
                'image_url': link
            })

            # Save metadata to JSON file
            json_path = os.path.join('unsplash_images', f'image_{num}.json')
            with open(json_path, 'w') as json_file:
                json.dump(metadata, json_file, indent=4)

            print(f"Metadata for {img_name} saved")
        else:
            print(f"Failed to download {link}: Status code {response.status_code}")
    except Exception as e:
        print(f"Error occurred while downloading {link}: {e}")

print("Metadata JSON files created")



Downloaded image_0.jpg
Failed to get tags for unsplash_images\image_0.jpg: Status code 403
Metadata for image_0.jpg saved
Downloaded image_1.jpg
Failed to get tags for unsplash_images\image_1.jpg: Status code 403
Metadata for image_1.jpg saved
Downloaded image_2.jpg
Failed to get tags for unsplash_images\image_2.jpg: Status code 403
Metadata for image_2.jpg saved


KeyboardInterrupt: 